#Setting up


In [ ]:
from skimage.filters import threshold_otsu
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import active_contour
from skimage import io, color, measure, segmentation
import os
import pycocotools.mask as mask_util
from skimage import io, segmentation, color, draw
import torch
import pickle
import scipy.io as sio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Set location of batch 1 dataset, train or test and saving path

In [ ]:
# Change dataset_path the location of the dataset
dataset_path = "/content/drive/MyDrive/AAR/seedsegment"
#Change for train or test set
img_path = "test"
#Change for saving location of the segmentations (the following will save the file at the specified location as detections.npy; default file name needs to be detections.npy)
save_path = "/content/detections"

#Defined functions for performing segmentation

In [ ]:
def segmentSeedPart(img, germinated):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  cv2_imshow(gray)
  _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
  cv2_imshow(thresh)

  mask = cv2.bitwise_not(thresh)
  cv2_imshow(mask)
  cv2_imshow(cv2.bitwise_or(mask, germinated))
  cv2_imshow(cv2.bitwise_not(cv2.bitwise_or(mask, germinated)))
  cv2_imshow(cv2.bitwise_and(img, img, mask=cv2.bitwise_or(mask, germinated)))
  mask = cv2.bitwise_and(mask, cv2.bitwise_not(germinated))
  cv2_imshow(mask)
 
  contours = measure.find_contours(mask, 0.5)

  longest=0
  for contour in contours:
      if len(contour)>longest:
        init = contour
        longest=len(contour)


  gray = color.rgb2gray(img)
  snake = segmentation.active_contour(gray, init, alpha=0.5, beta=1.0, gamma=0.01, max_num_iter=500)

  new_mask = np.zeros(gray.shape, dtype=bool)
  rr, cc = draw.polygon(snake[:, 0], snake[:, 1])
  new_mask[rr, cc] = 1

  return new_mask.astype(np.uint8)*255

In [ ]:
def segmentGerminated(img):
  hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  saturation = hsv_img[:, :, 1]
  cv2_imshow(saturation)
  thresh_value = threshold_otsu(saturation)
  saturation_binary_mask = saturation >= thresh_value
  saturation_binary_mask = saturation_binary_mask.astype(np.uint8)*255
  cv2_imshow(saturation_binary_mask)
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
  saturation_binary_mask = cv2.morphologyEx(saturation_binary_mask, cv2.MORPH_OPEN, kernel)
  cv2_imshow(saturation_binary_mask)
  contours = measure.find_contours(saturation_binary_mask, 0.5)

  longest=0
  for contour in contours:
      if len(contour)>longest:
        init = contour
        longest=len(contour)

  new_mask = np.zeros(saturation_binary_mask.shape, dtype=bool)
  rr, cc = draw.polygon(init[:, 0], init[:, 1])
  new_mask[rr, cc] = 1
  new_mask = new_mask

  return new_mask.astype(np.uint8)*255

In [ ]:
def calculateSegmentation(img):
  germinated_part=segmentGerminated(img)
  cv2_imshow(germinated_part)
  seed_mask=segmentSeedPart(img, germinated_part)
  cv2_imshow(seed_mask)

  global_mask = cv2.bitwise_or(germinated_part, seed_mask)
  cv2_imshow(global_mask)

  return global_mask, germinated_part, seed_mask

In [ ]:
def calculatebbox(binary):
  start_row_value = 0
  start_column_value = 0

  end_row_value = 0
  end_column_value = 0
  bin_height, bin_width = binary.shape

  for row in range(bin_height):
    
    for column in range(bin_width):
      if binary[row, column]:
        start_row_value = row
        break
    if start_row_value>0:
      break

  for column in range(bin_width):
    for row in range(bin_height):
      if binary[row, column]:
        start_column_value = column
        break
    if start_column_value>0:
      break

  for row in range(bin_height):
    for column in range(bin_width):
      if binary[(bin_height-1) - row, column]:

        end_row_value = (bin_height-1) - row
        break
    if end_row_value>0:
      break

  for column in range(bin_width):
    for row in range(bin_height):
      if binary[row, (bin_width-1) - column]:

        end_column_value = (bin_width-1) - column
        break
    if end_column_value>0:
      break

  return [start_column_value, start_row_value, end_column_value, end_row_value]

#Perform segmentation
After setting up the path of the dataset, indicating train or test set and the saving path, run the defined function above, and then the following to run segmentation.
The resulting file is a .npy file. 

In [ ]:
relative_path ="datasets/batch1seed"
seed_list=[]
id=0
class_name = "batch1seed"
class_id = 13
good_seed_dir = os.path.join(dataset_path, img_path, "GoodSeed")
bad_seed_dir = os.path.join(dataset_path, img_path, "BadSeed")
num_parts = 2
score = 0.9

for goodseed in os.listdir(good_seed_dir):
  print("Reading seed: ", goodseed, " ID: ", id)
  seed_true_path = os.path.join(good_seed_dir, goodseed)

  seed_retrieve_path = os.path.join(relative_path,  img_path, "GoodSeed", goodseed)

  seed_img = cv2.imread(seed_true_path)

  global_mask, germinated_part, seed_mask = calculateSegmentation(seed_img)
  cv2_imshow(cv2.bitwise_and(seed_img, seed_img, mask = global_mask))
  cv2_imshow(cv2.bitwise_and(global_mask, cv2.bitwise_not(germinated_part)))
  cv2_imshow(seed_img)
  cv2_imshow(global_mask)
  cv2_imshow(germinated_part)
  cv2_imshow(seed_mask)

  global_bbox = calculatebbox(global_mask)
  germinated_bbox = calculatebbox(germinated_part)
  seed_bbox = calculatebbox(seed_mask)

  img_height, img_width = global_mask.shape

  seed_semantics = []
  part_class_id = 0

  seed_semantics.append({
      'class':'germinated',
      'class_id':part_class_id,
      'bbox': torch.Tensor(germinated_bbox).numpy(),
      'score':score,
      'frequency':score,
      'mask': mask_util.encode(np.asfortranarray(germinated_part)),
  })

  part_class_id = 1
  seed_semantics.append({
      'class':'seed_body',
      'class_id':part_class_id,
      'bbox': torch.Tensor(seed_bbox).numpy(),
      'mask': mask_util.encode(np.asfortranarray(seed_mask)),
      'score':score,
      'frequency':score,
  })

  seed_list.append({
      'id': id,
      'class': class_name,
      'class_id': class_id,
      'image_height': img_height,
      'image_width': img_width,
      'image_path': seed_retrieve_path,
      'score':score,
      'bbox': torch.Tensor(global_bbox).numpy(),
      'mask': mask_util.encode(np.asfortranarray(global_mask)),
      'num_parts': len(seed_semantics),
      'parts': seed_semantics,
  })
                
  id += 1

for badseed in os.listdir(bad_seed_dir):
  print("Reading seed: ", badseed, " ID: ", id)

  seed_true_path = os.path.join(bad_seed_dir, badseed)

  seed_retrieve_path = os.path.join(relative_path,  img_path, "BadSeed", badseed)

  seed_img = cv2.imread(seed_true_path)
  global_mask, germinated_part, seed_mask = calculateSegmentation(seed_img)
  global_bbox = calculatebbox(global_mask)
  germinated_bbox = calculatebbox(germinated_part)
  seed_bbox = calculatebbox(seed_mask)

  img_height, img_width = global_mask.shape

  seed_semantics = []

  part_class_id = 0

  seed_semantics.append({
      'class':'germinated',
      'class_id':part_class_id,
      'score':score,
      'frequency':score,
      'bbox': torch.Tensor(germinated_bbox).numpy(),
      'mask': mask_util.encode(np.asfortranarray(germinated_part)),
  })

  part_class_id = 1
  seed_semantics.append({
      'class':'seed_body',
      'class_id':part_class_id,
      'score':score,
      'frequency':score,
      'bbox': torch.Tensor(seed_bbox).numpy(),
      'mask': mask_util.encode(np.asfortranarray(seed_mask)),
  })

  seed_list.append({
      'id': id,
      'class': class_name,
      'class_id': class_id,
      'image_height': img_height,
      'image_width': img_width,
      'image_path': seed_retrieve_path,
      'score':score,
      'bbox': torch.Tensor(global_bbox).numpy(),
      'mask': mask_util.encode(np.asfortranarray(global_mask)),
      'num_parts': len(seed_semantics),
      'parts': seed_semantics,
  })
                
  id += 1

np.save(save_path, seed_list)